In [3]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [4]:
sample = True
data_dir = 'data/m4a/'
features = {
    'emotion_embeddings': ['id_gems.tsv'],
    'audio_embeddings': ['id_musicnn.tsv'],
    'textual_embeddings': ['id_bert.tsv'],
    'visual_embeddings': ['id_resnet.tsv']
}

In [5]:
if sample:
    output_dir = 'data/sample/'
    df_interactions = pd.read_csv('data/m4a/userid_trackid_timestamp.tsv', sep='\t', nrows=100000)
else:
    output_dir = 'data/m4a/'
    df_interactions = pd.read_csv('data/m4a/userid_trackid_timestamp.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)

df_interactions.head()

,user_id,track_id,timestamp,rating
0,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:42:38,5
1,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:38:53,5
2,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:35:08,5
3,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:31:23,5
4,51549,iJTBIGHPjgJcT4Bt,2013-01-27 21:27:38,5


In [6]:
for feature_type in features:
    for file in features[feature_type]:
        print('Processing', file)
        df = pd.read_csv(os.path.join(data_dir, file), index_col=0, sep='\t')
        df = df[df.index.isin(sample_items)]

        feature_name = file.split('.')[0].split('_')[1]
        if sample:
            df = df.reindex(sample_items).dropna()
            folder_path = os.path.join('data/sample/', feature_type, feature_name)
        else:
            folder_path = os.path.join(data_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            np.save(os.path.join(folder_path, str(index) + '.npy'), row.values)

Processing id_gems.tsv


14270it [00:36, 392.76it/s]


Processing id_musicnn.tsv


14270it [00:35, 398.30it/s]


Processing id_bert.tsv


14270it [00:37, 378.72it/s]


Processing id_resnet.tsv


12879it [00:38, 336.57it/s]
